<a href="https://colab.research.google.com/github/RickBerends/Domain-Specific-BERT-for-Depression-Detection/blob/main/WorkingElmo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import torch
import tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import glob

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
os.chdir('/content/drive/My Drive/Splits')

#Load train, split = 0 for train. 
df = pd.read_csv("train_split.csv", usecols=[0,1])
df['split'] = 0
df['Participant_ID'].astype(int)

#Load test split, split = 2 for test.
df1 = pd.read_csv("full_test_split copy.csv", sep=';', usecols=[0,1])
df1['split'] = 2
df1['Participant_ID'].astype(int)
df1.rename(columns = {'PHQ_Binary':'PHQ8_Binary'}, inplace = True)

#Load dev split, split = 1 for dev.
df2 = pd.read_csv("dev_split.csv", usecols=[0,1])
df2['split'] = 1
df2['Participant_ID'].astype(int)

#Get them together
final_df = pd.concat([df,df1,df2])
final_df.sort_values('Participant_ID', inplace=True)

#Building data cleaning. 

import nltk
nltk.download('popular')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re

def clean_text(text): 
  #stop_words = set(stopwords.words('english'))
  #lemmatizer = WordNetLemmatizer()

  text = text.lower()
  text = re.sub(r"<sync>", "",text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"there's", "there is", text)
  text = re.sub(r"we're", "we are", text)
  text = re.sub(r"That's", "That is", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"they're", "they are", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"wasn't", "was not", text)
  text = re.sub(r"don't", "do not", text)
  text = re.sub(r"aren't", "are not", text)
  text = re.sub(r"isn't", "is not", text)
  text = re.sub(r"what's", "what is", text)
  text = re.sub(r"haven't", "have not", text)
  text = re.sub(r"hasn't", "has not", text)
  text = re.sub(r"there's", "there is", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"you're", "you are", text)
  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"shouldn't", "should not", text)
  text = re.sub(r"wouldn't", "would not", text)
  text = re.sub(r"isn't", "is not", text)
  text = re.sub(r"here's", "here is", text)
  text = re.sub(r"you've", "you have", text)
  text = re.sub(r"we're", "we are", text)
  text = re.sub(r"what's", "what is", text)
  text = re.sub(r"couldn't", "could not", text)
  text = re.sub(r"we've", "we have", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"doesn't", "does not", text)
  text = re.sub(r"who's", "who is", text)
  text = re.sub(r"y'all", "you all", text)
  text = re.sub(r"would've", "would have", text)
  text = re.sub(r"it'll", "it will", text)
  text = re.sub(r"we'll", "we will", text)
  text = re.sub(r"we've", "we have", text)
  text = re.sub(r"he'll", "he will", text)
  text = re.sub(r"y'all", "you all", text)
  text = re.sub(r"weren't", "Were not", text)
  text = re.sub(r"didn't", "did not", text)
  text = re.sub(r"they'll", "they will", text)
  text = re.sub(r"they'd", "they would", text)
  text = re.sub(r"don't", "do not", text)
  text = re.sub(r"they've", "they have", text)
  text = re.sub(r"i'd", "I would", text)
  text = re.sub(r"should've", "should have", text)
  text = re.sub(r"where's", "where is", text)
  text = re.sub(r"we'd", "we would", text)
  text = re.sub(r"i'll", "i will", text)
  text = re.sub(r"weren't", "were not", text)
  text = re.sub(r"they're", "they are", text)
  text = re.sub(r"let's", "let us", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"don't", "do not", text)
  text = re.sub(r"you're", "you are", text)
  text = re.sub(r"i've", "i have", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"i'll", "i will", text)
  text = re.sub(r"doesn't", "does not", text)
  text = re.sub(r"i'd", "i would", text)
  text = re.sub(r"didn't", "did not", text)
  text = re.sub(r"ain't", "am not", text)
  text = re.sub(r"you'll", "you will", text)
  text = re.sub(r"i've", "i have", text)
  text = re.sub(r"Don't", "do not", text)
  text = re.sub(r"i'll", "i will", text)
  text = re.sub(r"i'd", "i would", text)
  text = re.sub(r"let's", "Let us", text)
  text = re.sub(r"you'd", "you would", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"ain't", "am not", text)
  text = re.sub(r"haven't", "have not", text)
  text = re.sub(r"could've", "could have", text)
  text = re.sub(r"youve", "you have", text)  
  text = re.sub(r"[-()\"#/@;:<>.?,]","",text)
  text = ' '.join(text.split())
  text = text.split(' ')
  #text = [w for w in text if not w in stop_words]
  #text = [lemmatizer.lemmatize(w) for w in text]
  text = ' '.join(text)
  return text

#Loading all the text from the individual transcripts into one dataframe. 
os.chdir('/content/drive/My Drive/DAICWOZ_transcripts')

lst = []
labels = []
pd.set_option('max_colwidth', 800)

for file in glob.glob("*.csv"):
    name = file
    try: 
        df = pd.read_csv(name, usecols=[2,3], sep='\t')
        outcome = df['speaker'] == 'Participant' 
        new_str = ''
    except ValueError: 
        print(name)
        continue
    for index, j in enumerate(outcome):
        if j == False: 
            continue
        else: 
            response = df['value'][index:index+1]
            response = response.to_string(index=False)
            new_str+= response
    new_str = clean_text(new_str)
    lst.append(new_str)
    labels.append(name.split('_')[0])

final_df['text'] = lst

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [ ]:
#Split the text, based on the length of the text. 

def get_split(text1): 
  l_total = []
  l_partial = []

  #Check whether or not the text is bigger than 150 words and should be split.
  if len(text1.split())//20>0:
    n = len(text1.split())//20
  else: 
    n = 1
  
  #If that's the case, the first split is smaller than 200, the next split is [150:350], [300:550] etc. 
  for w in range(n): 
    if w == 0: 
      l_partial = text1.split()[:20]
      l_total.append(" ".join(l_partial))

    else: 
      l_partial = text1.split()[w*20: w*20+ 20]
      l_total.append(" ".join(l_partial))
  return l_total

twohundred_lst = []

for i in lst: 
  twohundred_lst.append(get_split(i))

#Finally replace the "text" with text, split up in lists.
final_df['text'] = twohundred_lst

In [ ]:
df_train = pd.DataFrame()
df_val = pd.DataFrame()
df_test = pd.DataFrame()

df_train = final_df[final_df['split'] == 0]
df_val = final_df[final_df['split'] == 1]
df_test = final_df[final_df['split'] == 2]

In [ ]:
random_seed = 42

#Copy dataframe to new dataframe
df_train_over = df_train.copy()

#Sample random rows
n = 77 - 30 

#Minority class
minor = df_train[df_train['PHQ8_Binary'] == 1]

rows = minor.sample(47, replace = True)

over_train_df = df_train_over.append(rows, ignore_index=True)
df_train = over_train_df

In [ ]:
!pip install "tensorflow>=2.0.0"
!pip install --upgrade tensorflow-hub

Requirement already up-to-date: tensorflow-hub in /usr/local/lib/python3.7/dist-packages (0.12.0)


In [ ]:
df_train.text[1][1]

'it uh just unemployed at the moment but uh actively seeking uh uh doing what i am supposed to be'

In [ ]:
# import necessary libraries
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.get_logger().setLevel('ERROR')
  
# Load pre trained ELMo model
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

def get_elmo(dataframe):

    """ This function takes a list of interview data, where every part of the
    list is one interview, that consists of multiple sentences

    returns: elmo embedding of every sentence in the list
    """
    elmo_embeddings = []

    participants = dataframe["Participant_ID"]

    # For every participants...
    for i, p in enumerate(participants):

      """if i < 67: 
        continue
      else: """

      lst = [] 

      if i < 3: 
        continue
      else: 

        print("Running... {} / {}".format(i + 1, len(participants)))
        print("    Currently extracting elmo for participant {}".format(p))
    
        # Take the data from the dataframe
        transcript = dataframe.text.values[i]

        for j in range(0, 128): 
          try: 
            embeddings = elmo(
              [
                df_train.text.values[i][j],
              ],
            signature="default",
            as_dict=True)["default"]
            init = tf.initialize_all_variables()
            sess = tf.Session()
            sess.run(init)

            one = sess.run(embeddings[0])
            temporary_lst = one.tolist()
            lst.append(temporary_lst)
            
          except:
              continue

        # Save to file!
        filename = str(p) + "_" + str(i) + "_elmo.csv"
        file_path = "/content/drive/My Drive/elmo_train_1/" + filename
        np.savetxt(file_path, lst, delimiter=',', fmt='%s')
    
    return elmo_embeddings

# References:
    # - https://github.com/allenai/allennlp/blob/v0.9.0/tutorials/how_to/elmo.md
    # - https://github.com/RicherMans/text_based_depression

In [ ]:
get_elmo(df_train)

Running... 4 / 107
    Currently extracting elmo for participant 310
Running... 5 / 107
    Currently extracting elmo for participant 312
Running... 6 / 107
    Currently extracting elmo for participant 313
Running... 7 / 107
    Currently extracting elmo for participant 315
Running... 8 / 107
    Currently extracting elmo for participant 316
Running... 9 / 107
    Currently extracting elmo for participant 317
Running... 10 / 107
    Currently extracting elmo for participant 318
Running... 11 / 107
    Currently extracting elmo for participant 319
Running... 12 / 107
    Currently extracting elmo for participant 320
Running... 13 / 107
    Currently extracting elmo for participant 321


In [ ]:
get_elmo(df_val)
#Check how 377 looks.

Running... 7 / 35
    Currently extracting elmo for participant 377
    # of characters: 8
['synch sniffle sniffle yes feeling okay little bit tired part feeling pretty good actually originally woodland hill san fernando valley born moved lomita los angeles county mm probably around nineteen ninety three mean twenty three yeah nineteen year living yeah clear throat thing really like would say know beach um know much traffic really care um definitely know kinda like nightlife um know people general like conversing others yeah clear throat right yep yeah actually mean cannot see shoe um yeah actually travel lot place like hawaii caribbean uh let u see make sound mouth um know different place like arizona oregon washington pretty much would say usually mainly like um hawaii caribbean mexico well yep clear throat um think experience know like cannot kinda experience culture um different people know whole background scenery know something new experience would say something know people seen 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


1
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


5
1024
Running... 35 / 35
    Currently extracting elmo for participant 492
    # of characters: 5
['yes pretty good laughter oklahoma mhm yup oklahoma city uh weather great love near ocean palm tree uh mountain love laughter um pretty easy difference huge city compared traffic definitely adjustment laughter um always wanted live california uh hated landlocked oklahoma wanted get west coast near ocean studied psychology minored art um moment um well actually think misunderstood question still working degree working field psychology well got hired working group home teenage boy trouble law dream job oh really know think wanna help people however um like working people realm psychology think may already dream job laughter relax like read watch movie um like go beach watch sunset um hang friend thing make really mad um probably injustice towards people um people mistreated abused um people lied deceived including hate lied laughter annoyed uh depends situation try like count ten take deep

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


1
2
3
4
5
1024


[]

In [ ]:
get_elmo(df_test)

Running... 1 / 47
    Currently extracting elmo for participant 300
    # of characters: 3
["yes well los angeles greater los angeles area people diversity various entertainment activity um fickle weather traffic litter traveled domestically internationally seeing place people live culture last trip went san diego uh childhood adult um family vacation um baja rosarito beach yeah reason memory come mind um weekend easter usually go easter east easter spring break rosarito baja um bachelor's b bachelor's communication study open field deal um communication interactively performing art telecommunication business entrepreneurship actually unemployed um think business dream dream job company um would like open something like telecommunication area business something online online service um like make thing hand next venture would would create natural soy candle something area um mixture mixture shy outgoing observe people crowd usually make determination remain reserved remain open listen l

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


1
2
3
4
5
1024


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Running... 2 / 47
    Currently extracting elmo for participant 301
    # of characters: 2
["yes alright los angeles california yep um light big city always something going traffic uh business administration business management uh something else networking moment sigh hmm uh open big clothing line supply whole world clothing laughter yeah definitely uh 'cause laughter often time yeah lied guess people think dumber laughter uh try get away everything go space well great controlling temper sigh um probably like two day ago laughter probably sport real argument sharp inhaling lot travel enough able see get change pace see somebody new see new face new environment mm let see last trip vega know vega go pretty fun lotta lotta drinking lot partying going laughter um pretty openminded kind pretty much everybody pretty evenkeel guy alright could better ma'am nope uh lately pretty tough guess staying late uh fault laughter know guess night owl sudden turning owl sluggish tired day sleep well da

...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


1
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2
3
4
5
1024


[]

In [ ]:
os.chdir('/content/drive/My Drive/elmo')
i = 0
for file in glob.glob("*.csv"):
  name = file
  print(name)

303_elmo.csv
304_elmo.csv
305_elmo.csv
310_elmo.csv
312_elmo.csv
313_elmo.csv
315_elmo.csv
316_elmo.csv
317_elmo.csv
318_elmo.csv
322_elmo.csv
324_elmo.csv
325_elmo.csv
326_elmo.csv
327_elmo.csv
328_elmo.csv
333_elmo.csv
336_elmo.csv
340_elmo.csv
341_elmo.csv
343_elmo.csv
345_elmo.csv
352_elmo.csv
357_elmo.csv
358_elmo.csv
360_elmo.csv
363_elmo.csv
364_elmo.csv
366_elmo.csv
368_elmo.csv
369_elmo.csv
370_elmo.csv
371_elmo.csv
374_elmo.csv
375_elmo.csv
379_elmo.csv
380_elmo.csv
383_elmo.csv
385_elmo.csv
386_elmo.csv
391_elmo.csv
392_elmo.csv
393_elmo.csv
397_elmo.csv
400_elmo.csv
401_elmo.csv
402_elmo.csv
409_elmo.csv
412_elmo.csv
415_elmo.csv
416_elmo.csv
419_elmo.csv
423_elmo.csv
425_elmo.csv
427_elmo.csv
428_elmo.csv
429_elmo.csv
430_elmo.csv
433_elmo.csv
434_elmo.csv
437_elmo.csv
443_elmo.csv
444_elmo.csv
445_elmo.csv
446_elmo.csv
447_elmo.csv
449_elmo.csv
454_elmo.csv
455_elmo.csv
456_elmo.csv
457_elmo.csv
463_elmo.csv
464_elmo.csv
468_elmo.csv
471_elmo.csv
473_elmo.csv
474_elmo.csv

In [ ]:
df = pd.read_csv('303_elmo.csv', header=None, delimiter='delimiter', index_col = False)
array = df.to_numpy()
lst = []
lst.append(array[0][0])
lst.append(array[1][0])
lst.append(array[2][0])
lst.append(array[3][0])
lst.append(array[4][0])

lst[4]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


'[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [ ]:
import re
print(re.sub('[\[\]]', '', np.array_str(a)))

In [ ]:
lst = []
lst.append(df.values.tolist())

In [ ]:
dataframe_lst = []
dataframe_lst.append(df)

dataframe_lst

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (5,1024)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 4, batch_size = 32)

In [ ]:
two.shape

(200, 1024)

In [ ]:
zsum = 0
ztotal = 0 

for i, j in enumerate(df_train.Participant_ID.values): 
  transcript = df_train.text.values[i]
  #print(len(transcript))
  ztotal+=1
  zsum+=len(transcript)

print(zsum)
print(ztotal)
print(zsum/ztotal)

648
154
4.207792207792208
